<a href="https://colab.research.google.com/github/parashuramkh/Crop_recomendation/blob/main/Parashuram_Final_Crop_Recommendation_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import urllib.request
import json
from flask import Flask, request, render_template_string
from pyngrok import ngrok
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score, mean_absolute_error
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import label_binarize
from imblearn.over_sampling import SMOTE

In [ ]:
app = Flask(__name__)

In [ ]:

# Step 1: Load Data
# Load datasets
pin_data = pd.read_csv('C://Users//paras//Downloads//Final Production data//FF//PIN.csv', encoding = 'latin1')
apc_data = pd.read_csv('C://Users//paras//Downloads//Final Production data//FF//APC.csv', encoding = 'latin1')

In [ ]:
apc_data.head()

,Country,Location_name,Region,Latitude,Longitude,Avg_temp,total_rainfall,Avg_humidity,Crop,Season,Average_Yield
0,India,Ashoknagar,Madhya Pradesh,24.57,77.72,90.918941,18.9,67,Wheat,Rabi,1050
1,India,Ashoknagar,Madhya Pradesh,24.57,77.72,95.208941,18.9,71,Wheat,Rabi,1050
2,India,Ashoknagar,Madhya Pradesh,24.57,77.72,87.408941,18.9,81,Wheat,Rabi,1050
3,India,Ashoknagar,Madhya Pradesh,24.57,77.72,101.948941,18.9,77,Wheat,Rabi,1050
4,India,Ashoknagar,Madhya Pradesh,24.57,77.72,74.578941,18.9,73,Wheat,Rabi,1050


In [ ]:
# Step 2: Preprocess Data
# Preprocess data
def preprocess_data(apc_data):
    apc_data.fillna(apc_data.mean(), inplace=True)
    return apc_data


In [ ]:
# Step 3: Feature Engineering
# Feature Engineering
def feature_engineering(apc_data):
    # Convert categorical variables to numerical (if applicable)
    apc_data['Crop'] = apc_data['Crop'].astype('category').cat.codes
    return apc_data

In [ ]:
print(pin_data.head(5))

   Pincode   Latitude  Longitude         Placename    District       StateName
0   515004  14.729960  77.573405  A Narayanapuram   ANANTHAPUR  Andhra Pradesh
1   515731  14.808274  77.539013          Akuledu   ANANTHAPUR  Andhra Pradesh
2   515002  14.625068  77.539013          Alamuru   ANANTHAPUR  Andhra Pradesh
3   515766  14.323319  77.033418        Allapuram   ANANTHAPUR  Andhra Pradesh
4   515415  14.960898  78.030954            Aluru   ANANTHAPUR  Andhra Pradesh


In [ ]:
 apc_data.head()

,Country,Location_name,Region,Latitude,Longitude,Avg_temp,total_rainfall,Avg_humidity,Crop,Season,Average_Yield
0,India,Ashoknagar,Madhya Pradesh,24.57,77.72,90.918941,18.9,67,Wheat,Rabi,1050
1,India,Ashoknagar,Madhya Pradesh,24.57,77.72,95.208941,18.9,71,Wheat,Rabi,1050
2,India,Ashoknagar,Madhya Pradesh,24.57,77.72,87.408941,18.9,81,Wheat,Rabi,1050
3,India,Ashoknagar,Madhya Pradesh,24.57,77.72,101.948941,18.9,77,Wheat,Rabi,1050
4,India,Ashoknagar,Madhya Pradesh,24.57,77.72,74.578941,18.9,73,Wheat,Rabi,1050


In [ ]:
# Directly define X and y
X = apc_data.drop(['Crop', 'Average_Yield'], axis=1)  # Features
y = apc_data['Crop']  # Target variable

In [ ]:
X.head()

,Country,Location_name,Region,Latitude,Longitude,Avg_temp,total_rainfall,Avg_humidity,Season
0,India,Ashoknagar,Madhya Pradesh,24.57,77.72,90.918941,18.9,67,Rabi
1,India,Ashoknagar,Madhya Pradesh,24.57,77.72,95.208941,18.9,71,Rabi
2,India,Ashoknagar,Madhya Pradesh,24.57,77.72,87.408941,18.9,81,Rabi
3,India,Ashoknagar,Madhya Pradesh,24.57,77.72,101.948941,18.9,77,Rabi
4,India,Ashoknagar,Madhya Pradesh,24.57,77.72,74.578941,18.9,73,Rabi


In [ ]:
X = apc_data[['Latitude', 'Longitude', 'Avg_temp', 'Avg_humidity' ]]

In [ ]:
# Check if the assignment was successful
print("Features (X):")
print(X.head())  # Display the first few rows of X

Features (X):
   Latitude  Longitude    Avg_temp  Avg_humidity
0     24.57      77.72   90.918941            67
1     24.57      77.72   95.208941            71
2     24.57      77.72   87.408941            81
3     24.57      77.72  101.948941            77
4     24.57      77.72   74.578941            73


In [ ]:
print("\nTarget (y):")
print(y.head())  # Display the first few rows of y


Target (y):
0    Wheat
1    Wheat
2    Wheat
3    Wheat
4    Wheat
Name: Crop, dtype: object


In [ ]:
# Feature Selection
def feature_selection(X, y):
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Apply PCA for dimensionality reduction
    pca = PCA(n_components=5)
    X_pca = pca.fit_transform(X_scaled)

    # SelectKBest
    best_features = SelectKBest(score_func=f_classif, k=10)
    X_selected = best_features.fit_transform(X_pca, y)
    return X_selected

In [ ]:
print("\nTarget (y):")
print(y.head())  # Display the first few rows of y


Target (y):
0    Wheat
1    Wheat
2    Wheat
3    Wheat
4    Wheat
Name: Crop, dtype: object


In [ ]:
# Preprocess the data
from sklearn.preprocessing import LabelEncoder

# Assuming X is feature matrix and y is the target variable
le = LabelEncoder()
X_encoded = X.copy()
for col in X_encoded.columns:
    if X_encoded[col].dtype == 'object':
        X_encoded[col] = le.fit_transform(X_encoded[col])

In [ ]:
# Assuming X is feature matrix and y is the target variable
le = LabelEncoder()
y_encoded = le.fit_transform(y)  # Encode target variable

In [ ]:
print("\nTarget (y_encoded):")
# Display the first 5 elements of the enoded y is the target variable
print(y_encoded[:5])



Target (y_encoded):
[32 32 32 32 32]


In [ ]:
# Step 4: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

In [ ]:

# Step 4: Class Balancing
def balance_classes(X_train, y_train):
    smote = SMOTE(random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X_train, y_train)
    return X_resampled, y_resampled


In [ ]:
# Step 6: Hyperparameter Tuning
def tune_hyperparameters(X_train, y_train, model):
    if isinstance(model, RandomForestClassifier):
        param_grid = {
            'n_estimators': [100, 200],
            'max_depth': [None, 10, 20],
            'min_samples_split': [2, 5]
        }
    elif isinstance(model, GradientBoostingClassifier):
        param_grid = {
            'n_estimators': [100, 200],
            'learning_rate': [0.01, 0.1],
            'max_depth': [3, 5]
        }
    elif isinstance(model, SVC):
        param_grid = {
            'C': [0.1, 1, 10],
            'kernel': ['linear', 'rbf']
        }
    elif isinstance(model, KNeighborsClassifier):
        param_grid = {
            'n_neighbors': [3, 5, 7],
            'weights': ['uniform', 'distance']
        }
    elif isinstance(model, MLPClassifier):
        param_grid = {
            'hidden_layer_sizes': [(50,), (100,), (50, 50)],
            'activation': ['tanh', 'relu'],
            'max_iter': [500]
        }
    elif isinstance(model, DecisionTreeClassifier):
        param_grid = {
            'max_depth': [None, 10, 20],
            'min_samples_split': [2, 5]
        }

    grid_search = GridSearchCV(model, param_grid, cv=5)
    grid_search.fit(X_train, y_train)
    return grid_search.best_estimator_

In [ ]:
# Define the models
decision_tree = DecisionTreeClassifier()
random_forest = RandomForestClassifier()
svm = SVC(probability=True)
knn = KNeighborsClassifier()
ann = MLPClassifier(max_iter=500)
gradient_boosting = GradientBoostingClassifier()

In [ ]:
# Train the models
decision_tree.fit(X_train, y_train)
random_forest.fit(X_train, y_train)
svm.fit(X_train, y_train)
knn.fit(X_train, y_train)
ann.fit(X_train, y_train)
gradient_boosting.fit(X_train, y_train)

In [ ]:

# Predictions
rf_y_pred = random_forest.predict(X_test)

# Predicted probabilities for all classes
rf_y_prob = random_forest.predict_proba(X_test)

# Calculate metrics
rf_accuracy = accuracy_score(y_test, rf_y_pred)
rf_precision = precision_score(y_test, rf_y_pred, average='weighted')
rf_recall = recall_score(y_test, rf_y_pred, average='weighted')
rf_f1 = f1_score(y_test, rf_y_pred, average='weighted')
rf_mae = mean_absolute_error(y_test, rf_y_pred)

# AUC-ROC Score for multiclass classification
n_classes = rf_y_prob.shape[1]
y_test_bin = label_binarize(y_test, classes=np.arange(n_classes))

# Calculate AUC-ROC for each class
rf_auc_roc = roc_auc_score(y_test_bin, rf_y_prob, average='macro', multi_class='ovr')

# Print the results
print("Random Forest Model Performance:")
print("Accuracy:", rf_accuracy)
print("Precision:", rf_precision)
print("Recall:", rf_recall)
print("F1 Score:", rf_f1)
print("AUC-ROC Score:", rf_auc_roc)
print("Mean Absolute Error:", rf_mae)



Random Forest Model Performance:
Accuracy: 0.8233524747060432
Precision: 0.8253668830468073
Recall: 0.8233524747060432
F1 Score: 0.8243178110858637
AUC-ROC Score: 0.9948263161302419
Mean Absolute Error: 1.0619360131255127


In [ ]:
# This is a simple example; typically, you would compare it with training accuracy.
if rf_accuracy < 0.7:
    print("\nConclusion: The rf model might be underfitting.")
elif rf_accuracy > 0.95:
    print("\nConclusion: The rf model might be overfitting.")
else:
    print("\nConclusion: The rf model is likely well-balanced.")


Conclusion: The rf model is likely well-balanced.


In [ ]:
# Step 1: Make predictions using the Random Forest model
rf_y_pred = random_forest.predict(X_test)

# Step 2: Generate the confusion matrix
cm = confusion_matrix(y_test, rf_y_pred)

# Step 3: Get unique class labels
class_labels = np.unique(y_test)

# Step 4: Create a heatmap for better visualization
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False,
           xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix for Random Forest Model')
plt.show()

In [ ]:

# Make predictions
svm_y_pred = svm.predict(X_test)

# Get predicted probabilities for each class
svm_y_prob = svm.predict_proba(X_test)

# Calculate metrics
svm_accuracy = accuracy_score(y_test, svm_y_pred)
svm_precision = precision_score(y_test, svm_y_pred, average='weighted')
svm_recall = recall_score(y_test, svm_y_pred, average='weighted')
svm_f1 = f1_score(y_test, svm_y_pred, average='weighted')
svm_mae = mean_absolute_error(y_test, svm_y_pred)

# AUC-ROC Score for multiclass classification
n_classes = svm_y_prob.shape[1]
y_test_bin = label_binarize(y_test, classes=np.arange(n_classes))

# Calculate AUC-ROC for each class
svm_auc_roc = roc_auc_score(y_test_bin, svm_y_prob, average='macro', multi_class='ovr')

# Print the results
print("SVM Model Performance:")
print("Accuracy:", svm_accuracy)
print("Precision:", svm_precision)
print("Recall:", svm_recall)
print("F1 Score:", svm_f1)
print("AUC-ROC Score:", svm_auc_roc)
print("Mean Absolute Error:", svm_mae)

# Conclusion based on accuracy
if svm_accuracy < 0.7:
    print("\nConclusion: The SVM model might be underfitting.")
elif svm_accuracy > 0.95:
    print("\nConclusion: The SVM model might be overfitting.")
else:
    print("\nConclusion: The SVM model is likely well-balanced.")



SVM Model Performance:
Accuracy: 0.3828274541974296
Precision: 0.27041516182066344
Recall: 0.3828274541974296
F1 Score: 0.2612951251783918
AUC-ROC Score: 0.9048654180361113
Mean Absolute Error: 5.860678151490292

Conclusion: The SVM model might be underfitting.


C:\Users\paras\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# Step 1: Generate the confusion matrix for SVM
conf_matrix_svm = confusion_matrix(y_test, svm_y_pred)
conf_matrix_svm_df = pd.DataFrame(conf_matrix_svm, index=np.unique(y_test), columns=np.unique(y_test))

# Step 2: Calculate the total counts
TP = np.diagonal(conf_matrix_svm_df).sum()  # Sum of diagonal elements (True Positives)
FP = conf_matrix_svm_df.sum(axis=0).sum() - TP  # Sum of column sums minus TP (False Positives)
FN = conf_matrix_svm_df.sum(axis=1).sum() - TP  # Sum of row sums minus TP (False Negatives)
TN = conf_matrix_svm_df.values.sum() - (TP + FP + FN)  # Total sum minus TP, FP, FN (True Negatives)

# Step 3: Print the summary
print("Total Counts:")
print("True Positive (TP):", TP)  # Correct predictions of the positive class
print("True Negative (TN):", TN)  # Correct predictions of the negative class
print("False Positive (FP):", FP)  # Incorrect predictions of the negative class as positive
print("False Negative (FN):", FN)  # Incorrect predictions of the positive class as negative

In [ ]:

# Plot confusion matrix for SVM
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix_df, annot=True, fmt='d', cmap='Blues', cbar=False,
            annot_kws={"size": 16}, linewidths=.5)
plt.title('Confusion Matrix', fontsize=16)
plt.xlabel('Predicted Label', fontsize=14)
plt.ylabel('True Label', fontsize=14)
plt.show()

# ROC Curve for multiclass classification
plt.figure(figsize=(10, 8))
for i in range(n_classes):
    fpr, tpr, _ = roc_curve(y_test_bin[:, i], svm_y_prob[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, lw=2, label=f'Class {i} (area = %0.2f)' % roc_auc)

plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=14)
plt.ylabel('True Positive Rate', fontsize=14)
plt.title('Receiver Operating Characteristic (ROC) Curve', fontsize=16)
plt.legend(loc='lower right')
plt.show()



In [ ]:

# Make predictions
knn_y_pred = knn.predict(X_test)

# Get predicted probabilities for each class
knn_y_prob = knn.predict_proba(X_test)

# Calculate metrics
knn_accuracy = accuracy_score(y_test, knn_y_pred)
knn_precision = precision_score(y_test, knn_y_pred, average='weighted')
knn_recall = recall_score(y_test, knn_y_pred, average='weighted')
knn_f1 = f1_score(y_test, knn_y_pred, average='weighted')
knn_mae = mean_absolute_error(y_test, knn_y_pred)

# Print the results
print("KNN Model Performance:")
print("Accuracy:", knn_accuracy)
print("Precision:", knn_precision)
print("Recall:", knn_recall)
print("F1 Score:", knn_f1)
print("Mean Absolute Error:", knn_mae)



# Conclusion based on accuracy
if knn_accuracy < 0.7:
    print("\nConclusion: The KNN model might be underfitting.")
elif knn_accuracy > 0.95:
    print("\nConclusion: The KNN model might be overfitting.")
else:
    print("\nConclusion: The KNN model is likely well-balanced.")

KNN Model Performance:
Accuracy: 0.45720535958435876
Precision: 0.43592666795983837
Recall: 0.45720535958435876
F1 Score: 0.4405843694919504
Mean Absolute Error: 5.583128247197156

Conclusion: The KNN model might be underfitting.


C:\Users\paras\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:

# Plot Confusion Matrix
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix_df, annot=True, fmt='d', cmap='Blues', cbar=False,
            annot_kws={"size": 16}, linewidths=.5)
plt.title('Confusion Matrix', fontsize=16)
plt.xlabel('Predicted Label', fontsize=14)
plt.ylabel('True Label', fontsize=14)
plt.show()

# ROC Curve for multiclass classification
n_classes = knn_y_prob.shape[1]
y_test_bin = label_binarize(y_test, classes=np.arange(n_classes))

# Plot ROC curves for each class
plt.figure(figsize=(10, 8))
for i in range(n_classes):
    fpr, tpr, _ = roc_curve(y_test_bin[:, i], knn_y_prob[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, lw=2, label=f'Class {i} (area = %0.2f)' % roc_auc)

plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=14)
plt.ylabel('True Positive Rate', fontsize=14)
plt.title('Receiver Operating Characteristic (ROC) Curve', fontsize=16)
plt.legend(loc='lower right')
plt.show()

In [ ]:
# Step 1: Generate the confusion matrix for knn
conf_matrix_knn = confusion_matrix(y_test, knn_y_pred)
conf_matrix_knn_df = pd.DataFrame(conf_matrix_knn, index=np.unique(y_test), columns=np.unique(y_test))

# Step 2: Calculate the total counts
TP = np.diagonal(conf_matrix_knn_df).sum()  # Sum of diagonal elements (True Positives)
FP = conf_matrix_knn_df.sum(axis=0).sum() - TP  # Sum of column sums minus TP (False Positives)
FN = conf_matrix_knn_df.sum(axis=1).sum() - TP  # Sum of row sums minus TP (False Negatives)
TN = conf_matrix_knn_df.values.sum() - (TP + FP + FN)  # Total sum minus TP, FP, FN (True Negatives)

# Step 3: Print the summary
print("Total Counts for KNN:")
print("True Positive (TP):", TP)  # Correct predictions of the positive class
print("True Negative (TN):", TN)  # Correct predictions of the negative class
print("False Positive (FP):", FP)  # Incorrect predictions of the negative class as positive
print("False Negative (FN):", FN)  # Incorrect predictions of the positive class as negative

In [ ]:

# Make predictions for  ANN MODEL
ann_y_pred = ann.predict(X_test)

# Get predicted probabilities for each class
ann_y_prob = ann.predict_proba(X_test)

# Calculate metrics
ann_accuracy = accuracy_score(y_test, ann_y_pred)
ann_precision = precision_score(y_test, ann_y_pred, average='weighted')
ann_recall = recall_score(y_test, ann_y_pred, average='weighted')
ann_f1 = f1_score(y_test, ann_y_pred, average='weighted')
ann_mae = mean_absolute_error(y_test, ann_y_pred)

# Print the results
print("ANN Model Performance:")
print("Accuracy:", ann_accuracy)
print("Precision:", ann_precision)
print("Recall:", ann_recall)
print("F1 Score:", ann_f1)
print("Mean Absolute Error:", ann_mae)

# Conclusion based on accuracy
if ann_accuracy < 0.7:
    print("\nConclusion: The ANN model might be underfitting.")
elif ann_accuracy > 0.95:
    print("\nConclusion: The ANN model might be overfitting.")
else:
    print("\nConclusion: The ANN model is likely well-balanced.")


ANN Model Performance:
Accuracy: 0.43177467869838665
Precision: 0.41632015025271835
Recall: 0.43177467869838665
F1 Score: 0.3886092628042314
Mean Absolute Error: 5.79190593382554

Conclusion: The ANN model might be underfitting.


C:\Users\paras\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:

# Confusion Matrix
conf_matrix_ANN = confusion_matrix(y_test, ann_y_pred)
conf_matrix_ANN_df = pd.DataFrame(conf_matrix, index=np.unique(y_test), columns=np.unique(y_test))

# Step 1: Generate the confusion matrix for knn
conf_matrix_ann = confusion_matrix(y_test, ann_y_pred)
conf_matrix_ann_df = pd.DataFrame(conf_matrix_ann, index=np.unique(y_test), columns=np.unique(y_test))

# Step 2: Calculate the total counts
TP = np.diagonal(conf_matrix_ann_df).sum()  # Sum of diagonal elements (True Positives)
FP = conf_matrix_ann_df.sum(axis=0).sum() - TP  # Sum of column sums minus TP (False Positives)
FN = conf_matrix_ann_df.sum(axis=1).sum() - TP  # Sum of row sums minus TP (False Negatives)
TN = conf_matrix_ann_df.values.sum() - (TP + FP + FN)  # Total sum minus TP, FP, FN (True Negatives)

# Step 3: Print the summary
print("Total Counts for ANN:")
print("True Positive (TP):", TP)  # Correct predictions of the positive class
print("True Negative (TN):", TN)  # Correct predictions of the negative class
print("False Positive (FP):", FP)  # Incorrect predictions of the negative class as positive
print("False Negative (FN):", FN)  # Incorrect predictions of the positive class as negative

In [ ]:
# Plot Confusion Matrix  for ANN
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix_df, annot=True, fmt='d', cmap='Blues', cbar=False,
            annot_kws={"size": 16}, linewidths=.5)
plt.title('Confusion Matrix', fontsize=16)
plt.xlabel('Predicted Label', fontsize=14)
plt.ylabel('True Label', fontsize=14)
plt.show()

# ROC Curve for multiclass classification  for ANN
n_classes = ann_y_prob.shape[1]
y_test_bin = label_binarize(y_test, classes=np.arange(n_classes))

# Plot ROC curves for each class  for ANN
plt.figure(figsize=(10, 8))
for i in range(n_classes):
    fpr, tpr, _ = roc_curve(y_test_bin[:, i], ann_y_prob[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, lw=2, label=f'Class {i} (area = %0.2f)' % roc_auc)

plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=14)
plt.ylabel('True Positive Rate', fontsize=14)
plt.title('Receiver Operating Characteristic (ROC) Curve', fontsize=16)
plt.legend(loc='lower right')
plt.show()

In [ ]:
#gradient_boosting = GradientBoostingClassifier()

In [ ]:
#gradient_boosting.fit(X_train, y_train)

GradientBoostingClassifier()

In [ ]:
# Make predictions using the Gradient Boosting model
gb_y_pred = gradient_boosting.predict(X_test)  # Ensure this line is included

# Get predicted probabilities for each class (for multiclass classification)
gb_y_prob = gradient_boosting.predict_proba(X_test)

# Calculate metrics
gb_accuracy = accuracy_score(y_test, gb_y_pred)
gb_precision = precision_score(y_test, gb_y_pred, average='weighted')
gb_recall = recall_score(y_test, gb_y_pred, average='weighted')
gb_f1 = f1_score(y_test, gb_y_pred, average='weighted')

# AUC-ROC Score for multiclass classification
n_classes = gb_y_prob.shape[1]  # Get the number of classes from predicted probabilities
y_test_bin = label_binarize(y_test, classes=np.arange(n_classes))

# Calculate AUC-ROC for each class
gb_auc_roc = roc_auc_score(y_test_bin, gb_y_prob, average='macro', multi_class='ovr')

# Calculate Mean Absolute Error
gb_mae = mean_absolute_error(y_test, gb_y_pred)

# Print the results
print("Gradient Boosting Model Performance:")
print("Accuracy:", gb_accuracy)
print("Precision:", gb_precision)
print("Recall:", gb_recall)
print("F1 Score:", gb_f1)
print("AUC ROC:", gb_auc_roc)
print("Mean Absolute Error:", gb_mae)

# Conclusion based on accuracy
if gb_accuracy < 0.7:
    print("\nConclusion: The Gradient Boosting model might be underfitting.")
elif gb_accuracy > 0.95:
    print("\nConclusion: The Gradient Boosting model might be overfitting.")
else:
    print("\nConclusion: The Gradient Boosting model is likely well-balanced.")

Gradient Boosting Model Performance:
Accuracy: 0.8330598851517638
Precision: 0.8378480746012472
Recall: 0.8330598851517638
F1 Score: 0.8353009973424921
AUC ROC: 0.9956073977058291
Mean Absolute Error: 1.053732567678425

Conclusion: The Gradient Boosting model is likely well-balanced.


In [ ]:

# Confusion Matrix
conf_matrix_gb = confusion_matrix(y_test, gb_y_pred)
conf_matrix_gb_df = pd.DataFrame(conf_matrix_gb, index=np.unique(y_test), columns=np.unique(y_test))

# Step 1: Generate the confusion matrix
conf_matrix_gb = confusion_matrix(y_test, gb_y_pred)
conf_matrix_gb_df = pd.DataFrame(conf_matrix_gb, index=np.unique(y_test), columns=np.unique(y_test))

# Step 2: Calculate the total counts
TP = np.diagonal(conf_matrix_gb_df).sum()  # Sum of diagonal elements (True Positives)
FP = conf_matrix_gb_df.sum(axis=0).sum() - TP  # Sum of column sums minus TP (False Positives)
FN = conf_matrix_gb_df.sum(axis=1).sum() - TP  # Sum of row sums minus TP (False Negatives)
TN = conf_matrix_gb_df.values.sum() - (TP + FP + FN)  # Total sum minus TP, FP, FN (True Negatives)

# Step 3: Print the summary
print("Total Counts for Gradient Boosting:")
print("True Positive (TP):", TP)  # Correct predictions of the positive class
print("True Negative (TN):", TN)  # Correct predictions of the negative class
print("False Positive (FP):", FP)  # Incorrect predictions of the negative class as positive
print("False Negative (FN):", FN)  # Incorrect predictions of the positive class as negative

In [ ]:

# Plot Confusion Matrix for Gradient Boosting
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix_gb_df, annot=True, fmt='d', cmap='Blues', cbar=False,
            annot_kws={"size": 16}, linewidths=.5)
plt.title('Confusion Matrix for Gradient Boosting Model', fontsize=16)
plt.xlabel('Predicted Label', fontsize=14)
plt.ylabel('True Label', fontsize=14)
plt.show()

# ROC Curve for multiclass classification for Gradient Boosting
n_classes = gb_y_prob.shape[1]
y_test_bin = label_binarize(y_test, classes=np.arange(n_classes))

# Plot ROC curves for each class for Gradient Boosting
plt.figure(figsize=(10, 8))
for i in range(n_classes):
    fpr, tpr, _ = roc_curve(y_test_bin[:, i], gb_y_prob[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, lw=2, label=f'Class {i} (area = %0.2f)' % roc_auc)

plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=14)
plt.ylabel('True Positive Rate', fontsize=14)
plt.title('Receiver Operating Characteristic (ROC) Curve for Gradient Boosting Model', fontsize=16)
plt.legend(loc='lower right')
plt.show()


In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve, auc

# Make predictions for Decision Tree Model
dt_y_pred = decision_tree.predict(X_test)

# Get predicted probabilities for each class
dt_y_prob = decision_tree.predict_proba(X_test)

# Calculate metrics
dt_accuracy = accuracy_score(y_test, dt_y_pred)
dt_precision = precision_score(y_test, dt_y_pred, average='weighted')
dt_recall = recall_score(y_test, dt_y_pred, average='weighted')
dt_f1 = f1_score(y_test, dt_y_pred, average='weighted')

# Calculate AUC-ROC score, using 'ovr' (one-vs-rest) for multi-class classification
dt_auc_roc = roc_auc_score(y_test, dt_y_prob, multi_class='ovr')

dt_mae = mean_absolute_error(y_test, dt_y_pred)

# Print the results
print("Decision Tree Model Performance:")
print("Accuracy:", dt_accuracy)
print("Precision:", dt_precision)
print("Recall:", dt_recall)
print("F1 Score:", dt_f1)
print("AUC ROC:", dt_auc_roc)
print("Mean Absolute Error:", dt_mae)

# Conclusion based on accuracy
if dt_accuracy < 0.7:
    print("\nConclusion: The Decision Tree model might be underfitting.")
elif dt_accuracy > 0.95:
    print("\nConclusion: The Decision Tree model might be overfitting.")
else:
    print("\nConclusion: The Decision Tree model is likely well-balanced.")


NameError: name 'decision_tree' is not defined

In [ ]:

# Confusion Matrix
conf_matrix_dt = confusion_matrix(y_test, dt_y_pred)
conf_matrix_dt_df = pd.DataFrame(conf_matrix_dt, index=np.unique(y_test), columns=np.unique(y_test))

# Step 1: Generate the confusion matrix
conf_matrix_dt = confusion_matrix(y_test, dt_y_pred)
conf_matrix_dt_df = pd.DataFrame(conf_matrix_dt, index=np.unique(y_test), columns=np.unique(y_test))

# Step 2: Calculate the total counts
TP = np.diagonal(conf_matrix_dt_df).sum()  # Sum of diagonal elements (True Positives)
FP = conf_matrix_dt_df.sum(axis=0).sum() - TP  # Sum of column sums minus TP (False Positives)
FN = conf_matrix_dt_df.sum(axis=1).sum() - TP  # Sum of row sums minus TP (False Negatives)
TN = conf_matrix_dt_df.values.sum() - (TP + FP + FN)  # Total sum minus TP, FP, FN (True Negatives)

# Step 3: Print the summary
print("Total Counts for Decision Tree:")
print("True Positive (TP):", TP)  # Correct predictions of the positive class
print("True Negative (TN):", TN)  # Correct predictions of the negative class
print("False Positive (FP):", FP)  # Incorrect predictions of the negative class as positive
print("False Negative (FN):", FN)  # Incorrect predictions of the positive class as negative



In [ ]:
# Plot Confusion Matrix for Decision Tree
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix_dt_df, annot=True, fmt='d', cmap='Blues', cbar=False,
            annot_kws={"size": 16}, linewidths=.5)
plt.title('Confusion Matrix for Decision Tree Model', fontsize=16)
plt.xlabel('Predicted Label', fontsize=14)
plt.ylabel('True Label', fontsize=14)
plt.show()

# ROC Curve for multi-class classification for Decision Tree
plt.figure(figsize=(10, 8))

# Compute ROC curve and ROC area for each class
fpr = {}
tpr = {}
roc_auc = {}
for i in range(len(np.unique(y_test))):
    fpr[i], tpr[i], _ = roc_curve(y_test, dt_y_prob[:, i], pos_label=i)
    roc_auc[i] = auc(fpr[i], tpr[i])
    plt.plot(fpr[i], tpr[i], lw=2, label=f'Class {i} ROC curve (area = %0.2f)' % roc_auc[i])

plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=14)
plt.ylabel('True Positive Rate', fontsize=14)
plt.title('Receiver Operating Characteristic (ROC) Curve for Decision Tree Model', fontsize=16)
plt.legend(loc='lower right')
plt.show()


In [ ]:
# Create a dictionary to store the results
results = {
    'Decision Tree': {
        'accuracy': dt_accuracy,
        'precision': dt_precision,
        'recall': dt_recall,
        'f1_score': dt_f1,
        'auc_roc': dt_auc_roc,
        'mae': dt_mae
    },
    'Random Forest': {
        'accuracy': rf_accuracy,
        'precision': rf_precision,
        'recall': rf_recall,
        'f1_score': rf_f1,
        'auc_roc': rf_auc_roc,
        'mae': rf_mae
    },
    'SVM': {
        'accuracy': svm_accuracy,
        'precision': svm_precision,
        'recall': svm_recall,
        'f1_score': svm_f1,
        'auc_roc': svm_auc_roc,
        'mae': svm_mae
    },
    'KNN': {
        'accuracy': knn_accuracy,
        'precision': knn_precision,
        'recall': knn_recall,
        'f1_score': knn_f1,
        'mae': knn_mae
    },
    'ANN': {
        'accuracy': ann_accuracy,
        'precision': ann_precision,
        'recall': ann_recall,
        'f1_score': ann_f1,
        'mae': ann_mae
    },
    'Gradient Boosting': {
        'accuracy': gb_accuracy,
        'precision': gb_precision,
        'recall': gb_recall,
        'f1_score': gb_f1,
        'auc_roc': gb_auc_roc,
        'mae': gb_mae
    }
}


In [ ]:
# Create a DataFrame from the results dictionary
results_df = pd.DataFrame.from_dict(results, orient='index')

In [ ]:
print(results_df)

                   accuracy  precision    recall  f1_score   auc_roc       mae
Decision Tree      0.840306   0.840200  0.840306  0.840252  0.985259  0.956932
Random Forest      0.823352   0.825367  0.823352  0.824318  0.994826  1.061936
SVM                0.382827   0.270415  0.382827  0.261295  0.904865  5.860678
KNN                0.457205   0.435927  0.457205  0.440584       NaN  5.583128
ANN                0.431775   0.416320  0.431775  0.388609       NaN  5.791906
Gradient Boosting  0.833060   0.837848  0.833060  0.835301  0.995607  1.053733


In [ ]:
# Find the best model based on accuracy
best_model_name = results_df['accuracy'].idxmax()
best_accuracy = results_df.loc[best_model_name, 'accuracy']

# Print the best model and its accuracy
print(f"The best model is: {best_model_name}")
print(f"Accuracy of the best model: {best_accuracy:.4f}")

The best model is: Decision Tree
Accuracy of the best model: 0.8403


In [ ]:

def plot_roc_curve(model, X_test, y_test):
    if hasattr(model, 'predict_proba'):
        y_prob = model.predict_proba(X_test)
    else:
        raise ValueError('The model does not support probability predictions.')

    if y_prob.shape[1] == 2:  # Binary classification
        y_prob = y_prob[:, 1]  # Get probabilities for the positive class
        fpr, tpr, _ = roc_curve(y_test, y_prob)
        roc_auc = auc(fpr, tpr)

        plt.figure(figsize=(8, 6))
        plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (AUC = %0.2f)' % roc_auc)
        plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver Operating Characteristic (ROC) Curve')
        plt.legend(loc="lower right")
        plt.show()

    else:  # Multiclass classification
        from sklearn.preprocessing import label_binarize

        n_classes = y_prob.shape[1]
        y_test_bin = label_binarize(y_test, classes=np.arange(n_classes))

        plt.figure(figsize=(10, 8))
        for i in range(n_classes):
            fpr, tpr, _ = roc_curve(y_test_bin[:, i], y_prob[:, i])
            roc_auc = auc(fpr, tpr)
            plt.plot(fpr, tpr, lw=2, label=f'Class {i} (AUC = %0.2f)' % roc_auc)

        plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver Operating Characteristic (ROC) Curve')
        plt.legend(loc="lower right")
        plt.show()

# Example usage
# Assuming you have a trained model and test data
plot_roc_curve(best_model, X_test, y_test)


NameError: name 'best_model' is not defined

### the weather datafunction

In [ ]:
import urllib.request
import json
import numpy as np
import datetime
from sklearn.preprocessing import LabelEncoder


# Function to fetch current weather data from Visual Crossing
def fetch_weather_data(lat, lon, start_date, api_key):
    weather_api_query = f'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{lat},{lon}/{start_date}?unitGroup=us&key={api_key}&contentType=json'
    try:
        response = urllib.request.urlopen(weather_api_query)
        data = response.read()
        return json.loads(data.decode('utf-8'))
    except Exception as e:
        print(f"Error fetching weather data: {e}")
        return None

# Function to analyze weather data
def analyze_weather(weather_data):
    days = weather_data['days']
    if days:
        current_day = days[0]  # Assuming the first day in the list represents the current day
        temperature = current_day.get('temp', 'N/A')
        humidity = current_day.get('humidity', 'N/A')
        # Assuming pH is provided as humidity; adjust if necessary
        ph = current_day.get('humidity', 'N/A')
        rainfall = current_day.get('precip', 'N/A')
        return {
            'temperature': temperature,
            'humidity': humidity,
            'ph': ph,
            'rainfall': rainfall
        }
    else:
        return {'temperature': 'N/A', 'humidity': 'N/A', 'ph': 'N/A', 'rainfall': 'N/A'}

In [ ]:

# Function to get predictions from the model
def get_predictions(weather_data, model, label_encoder):
    # Create the feature array with explicit type conversion to float
    features = np.array([[float(weather_data['latitude']),
                          float(weather_data['longitude']),
                          float(weather_data['temperature']),
                          float(weather_data['humidity'])]])

    # Check for NaN values or all-zero features in the features array
    if np.isnan(features).any() or (features == 0).all(axis=1):
        print("Error: One or more feature values are NaN or all zero.")
        return []

    # Get predictions from the Gradient Boosting model
    predicted_indices = model.predict(features)

    # Decode the predicted indices to crop names
    predicted_crops = label_encoder.inverse_transform(predicted_indices)

    return predicted_crops

In [ ]:

# Assuming you have a label encoder for crops
from sklearn.preprocessing import LabelEncoder

# Create a label encoder and fit it on the crop data
label_encoder = LabelEncoder()
label_encoder.fit(y)  # y is your original crop labels

LabelEncoder()

In [ ]:
from datetime import datetime
import numpy as np

if __name__ == "__main__":
    # Prompt user for input
    user_pincode = input("Enter your pincode: ").strip()
    land_size = float(input("Enter the land size in acres: "))

    # Fetch location details based on pincode
    if user_pincode in pin_data['Pincode'].astype(str).values:  # Convert to string for comparison
        row = pin_data[pin_data['Pincode'] == int(user_pincode)].iloc[0]  # Ensure pincode is an integer
        latitude = row['Latitude']  # This is a float
        longitude = row['Longitude']  # This is a float

        # Check if the pincode exists in the DataFrame
        if int(user_pincode) in pin_data['Pincode'].values:
            # Fetching the corresponding row
            row = pin_data[pin_data['Pincode'] == int(user_pincode)].iloc[0]

            # Extracting the required fields
            place_name = row['Placename']
            district = row['District']
            state_name = row['StateName']

            # Printing the formatted output
            print(f"Your weather for the pincode {user_pincode}:")
            print(f"Place Name: {place_name}")
            print(f"District: {district}")
            print(f"State Name: {state_name}")

            # Fetch weather data based on user input
            api_key = 'DCDVS6HGLC8S6F657B22M9NNM'  # Replace with your actual API key
            start_date = datetime.today().strftime('%Y-%m-%d')  # Use the current date
            weather_data = fetch_weather_data(latitude, longitude, start_date, api_key)

            # Check if weather data was fetched successfully
            if weather_data is None:
                print("Error fetching weather data.")
                exit(1)

            # Analyze the fetched weather data
            analyzed_weather = analyze_weather(weather_data)
            # Line by line print of the weather data
            print(f"Temperature: {analyzed_weather['temperature']} °F")
            print(f"Humidity: {analyzed_weather['humidity']} %")
            print(f"pH: {analyzed_weather['ph']}")
            print(f"Rainfall: {analyzed_weather['rainfall']} inches")

            # Prepare weather data for predictions
            weather_data_for_prediction = {
                'latitude': latitude,
                'longitude': longitude,
                'temperature': analyzed_weather['temperature'],
                'humidity': analyzed_weather['humidity']
            }

            # Get predictions from the Gradient Boosting model
            predicted_crops = get_predictions(weather_data_for_prediction, gradient_boosting, label_encoder)

            # Define how many top crops you want to display
            top_n = 1

            print("\nTop Recommended crops:")

            # Ensure top_n does not exceed the number of predicted crops
            num_crops_to_display = min(top_n, len(predicted_crops))

            # Show only the top `num_crops_to_display` predicted crops or as many as are available
            for crop in predicted_crops[:num_crops_to_display]:
                print(crop)

            if len(predicted_crops) == 0:
                print("No crops predicted.")
            elif len(predicted_crops) < top_n:
                print(f"Fewer than {top_n} crops predicted. Showing all available.")

            # Insert your crop recommendation logic here
            if isinstance(predicted_crops, (list, np.ndarray)):
                for crop in predicted_crops[:num_crops_to_display]:
                    crop_data = apc_data[apc_data['Crop'] == crop]

                    if not crop_data.empty:
                        season = crop_data['Season'].values[0]
                        average_yield = crop_data['Average_Yield'].values[0]

                        # Calculate estimated production
                        estimated_production = land_size * average_yield

                        # Print the results with units in kilograms per acre
                        print(f"Best season for the crop {crop}: {season}")
                        print(f"Estimated per acre yield for {crop}: {average_yield} kgs/acre")
                        print(f"For your land size of {land_size} acres, the estimated production for {crop} is {estimated_production} kgs.")
                    else:
                        print(f"No information available for the crop {crop} at the given location.")
            else:
                print("Predicted crops should be a list or array of crop names.")

            # Thank you message in Latin
            print("Thank you.")
        else:
            print(f"Pincode {user_pincode} not found in the data.")
    else:
        print(f"Error: Pincode {user_pincode} not found.")
        exit(1)


Enter your pincode:  580025
Enter the land size in acres:  2


Your weather for the pincode 580025:
Place Name: Amragol 
District: Dharwad
State Name: Karnataka
Temperature: 73.3 °F
Humidity: 85.3 %
pH: 85.3
Rainfall: 0.088 inches

Top Recommended crops:
Urad
Best season for the crop Urad: Kharif
Estimated per acre yield for Urad: 600 kgs/acre
For your land size of 2.0 acres, the estimated production for Urad is 1200.0 kgs.
Thank you.


C:\Users\paras\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  warnings.warn(


In [ ]:
from datetime import datetime
# Main function to simulate user input and get predictions
if __name__ == "__main__":
    # Prompt user for input
    user_pincode = input("Enter your pincode: ").strip()
    land_size = float(input("Enter the land size in acres: "))

    # Fetch location details based on pincode
    if user_pincode in pin_data['Pincode'].astype(str).values:  # Convert to string for comparison
        row = pin_data[pin_data['Pincode'] == int(user_pincode)].iloc[0]  # Ensure pincode is an integer
        latitude = row['Latitude']  # This is a float
        longitude = row['Longitude']  # This is a float
    else:
        print(f"Error: Pincode {user_pincode} not found.")
        exit(1)  # Exit the program if the pincode is not found

    # Check if the pincode exists in the DataFrame
    if int(user_pincode) in pin_data['Pincode'].values:
        # Fetching the corresponding row
        row = pin_data[pin_data['Pincode'] == int(user_pincode)].iloc[0]

        # Extracting the required fields
        place_name = row['Placename']
        district = row['District']
        state_name = row['StateName']

        # Printing the formatted output
        print(f"Your weather for the pincode {user_pincode}:")
        print(f"Place Name: {place_name}")
        print(f"District: {district}")
        print(f"State Name: {state_name}")
    else:
        print(f"Pincode {user_pincode} not found in the data.")



    # Fetch weather data based on user input
    api_key = 'DCDVS6HGLC8S6F657B22M9NNM'  # Replace with your actual API key
    start_date = datetime.today().strftime('%Y-%m-%d')  # Use the current date
    weather_data = fetch_weather_data(latitude, longitude, start_date, api_key)



    # Check if weather data was fetched successfully
    if weather_data is None:
        print("Error fetching weather data.")
        exit(1)

    # Analyze the fetched weather data
    analyzed_weather = analyze_weather(weather_data)
     # Line by line print of the weather data
    print(f"Temperature: {analyzed_weather['temperature']} °F")
    print(f"Humidity: {analyzed_weather['humidity']} %")
    print(f"pH: {analyzed_weather['ph']}")
    print(f"Rainfall: {analyzed_weather['rainfall']} inches")


    # Prepare weather data for predictions
    weather_data_for_prediction = {
        'latitude': latitude,
        'longitude': longitude,
        'temperature': analyzed_weather['temperature'],
        'humidity': analyzed_weather['humidity']
    }

    # Get predictions from the Gradient Boosting model
    predicted_crops = get_predictions(weather_data_for_prediction, gradient_boosting, label_encoder)

    # Define how many top crops you want to display
    top_n = 1

    print("\nTop Recommended crops:")

    # Ensure top_n does not exceed the number of predicted crops
    num_crops_to_display = min(top_n, len(predicted_crops))

    # Show only the top `num_crops_to_display` predicted crops or as many as are available
    for crop in predicted_crops[:num_crops_to_display]:
        print(crop)

    if len(predicted_crops) == 0:
        print("No crops predicted.")
    elif len(predicted_crops) < top_n:
        print(f"Fewer than {top_n} crops predicted. Showing all available.")

    # Insert your crop recommendation logic here
    if isinstance(predicted_crops, (list, np.ndarray)):
        for crop in predicted_crops[:num_crops_to_display]:
            crop_data = apc_data[apc_data['Crop'] == crop]

            if not crop_data.empty:
                season = crop_data['Season'].values[0]
                average_yield = crop_data['Average_Yield'].values[0]

                # Calculate estimated production
                estimated_production = land_size * average_yield

                # Print the results with units in kilograms per acre
                print(f"Best season for the crop {crop}: {season}")
                print(f"Estimated per acre yield for {crop}: {average_yield} kgs/acre")
                print(f"For your land size of {land_size} acres, the estimated production for {crop} is {estimated_production} kgs.")
            else:
                print(f"No information available for the crop {crop} at the given location.")
    else:
        print("Predicted crops should be a list or array of crop names.")

    # Thank you message in Latin
    print("Thank you.")

Enter your pincode:  58
Enter the land size in acres:  2


Error: Pincode 58 not found.
Pincode 58 not found in the data.
Temperature: 73.3 °F
Humidity: 85.3 %
pH: 85.3
Rainfall: 0.088 inches

Top Recommended crops:
Urad
Best season for the crop Urad: Kharif
Estimated per acre yield for Urad: 600 kgs/acre
For your land size of 2.0 acres, the estimated production for Urad is 1200.0 kgs.
Thank you.


C:\Users\paras\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  warnings.warn(


In [ ]:
from datetime import datetime
# Main function to simulate user input and get predictions
if __name__ == "__main__":
    # Prompt user for input
    user_pincode = input("Enter your pincode: ").strip()
    land_size = float(input("Enter the land size in acres: "))

    # Fetch location details based on pincode
    if user_pincode in pin_data['Pincode'].astype(str).values:  # Convert to string for comparison
        row = pin_data[pin_data['Pincode'] == int(user_pincode)].iloc[0]  # Ensure pincode is an integer
        latitude = row['Latitude']  # This is a float
        longitude = row['Longitude']  # This is a float
    else:
        print(f"Error: Pincode {user_pincode} not found.")
        exit(1)  # Exit the program if the pincode is not found

    # Check if the pincode exists in the DataFrame
    if int(user_pincode) in pin_data['Pincode'].values:
        # Fetching the corresponding row
        row = pin_data[pin_data['Pincode'] == int(user_pincode)].iloc[0]

        # Extracting the required fields
        place_name = row['Placename']
        district = row['District']
        state_name = row['StateName']

        # Printing the formatted output
        print(f"Your weather for the pincode {user_pincode}:")
        print(f"Place Name: {place_name}")
        print(f"District: {district}")
        print(f"State Name: {state_name}")
    else:
        print(f"Pincode {user_pincode} not found in the data.")



    # Fetch weather data based on user input
    api_key = 'DCDVS6HGLC8S6F657B22M9NNM'  # Replace with your actual API key
    start_date = datetime.today().strftime('%Y-%m-%d')  # Use the current date
    weather_data = fetch_weather_data(latitude, longitude, start_date, api_key)



    # Check if weather data was fetched successfully
    if weather_data is None:
        print("Error fetching weather data.")
        exit(1)

    # Analyze the fetched weather data
    analyzed_weather = analyze_weather(weather_data)
     # Line by line print of the weather data
    print(f"Temperature: {analyzed_weather['temperature']} °F")
    print(f"Humidity: {analyzed_weather['humidity']} %")
    print(f"pH: {analyzed_weather['ph']}")
    print(f"Rainfall: {analyzed_weather['rainfall']} inches")


    # Prepare weather data for predictions
    weather_data_for_prediction = {
        'latitude': latitude,
        'longitude': longitude,
        'temperature': analyzed_weather['temperature'],
        'humidity': analyzed_weather['humidity']
    }

    # Get predictions from the Gradient Boosting model
    predicted_crops = get_predictions(weather_data_for_prediction, gradient_boosting, label_encoder)

    print("\nTop Recommended crops:")
    # Show only the top 3 predicted crops or as many as are available
    for crop in predicted_crops[:3]:
        print(crop)

    if len(predicted_crops) == 0:
        print("No crops predicted.")
    elif len(predicted_crops) < 3:
        print("Fewer than 3 crops predicted. Showing all available.")

    print("Thank you!")


Enter your pincode:  580025
Enter the land size in acres:  1


Your weather for the pincode 580025:
Place Name: Amragol 
District: Dharwad
State Name: Karnataka
Temperature: 73.3 °F
Humidity: 85.3 %
pH: 85.3
Rainfall: 0.088 inches

Top Recommended crops:
Urad
Fewer than 3 crops predicted. Showing all available.
Thank you!
Thank you!


C:\Users\paras\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  warnings.warn(


In [ ]:
apc_data.columns

Index(['Country', 'Location_name', 'Region', 'Latitude', 'Longitude',
       'Avg_temp', 'total_rainfall', 'Avg_humidity', 'Crop', 'Season',
       'Average_Yield'],
      dtype='object')